In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [4]:
df9 = df9.apply(pd.to_numeric, errors = 'coerce')
df9.describe().iloc[:, 14:]
# 없는 Column = depart, sunrise, sunset, codesum, station_nbr, date

,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,211788.000000,210012.000000,212454.000000,212454.000000,212010.000000,213120.000000,213120.000000,213120.000000,213120.000000,213120.0,213120.000000,213120.000000,213120.000000,213120.000000
mean,29.226908,30.001638,6.773041,21.450366,8.238848,0.486458,0.490625,0.179167,14.000000,9.0,56.000000,1.039077,2.993750,0.284375
std,0.207925,0.214619,3.950172,9.679790,3.726581,0.499818,0.499913,0.383493,10.000023,0.0,32.041715,10.315071,2.000516,0.451117
min,28.500000,29.270000,0.100000,1.000000,0.900000,0.000000,0.000000,0.000000,4.000000,9.0,1.000000,0.000000,0.000000,0.000000
25%,29.090000,29.850000,3.800000,14.000000,5.600000,0.000000,0.000000,0.000000,4.000000,9.0,28.000000,0.000000,1.000000,0.000000
50%,29.230000,29.990000,6.100000,21.000000,7.700000,0.000000,0.000000,0.000000,14.000000,9.0,56.000000,0.000000,3.000000,0.000000
75%,29.360000,30.140000,9.200000,30.000000,10.500000,1.000000,1.000000,0.000000,24.000000,9.0,84.000000,0.000000,5.000000,1.000000
max,29.980000,30.800000,22.800000,36.000000,22.500000,1.000000,1.000000,1.000000,24.000000,9.0,111.000000,390.000000,6.000000,1.000000


In [5]:
df9['store_nbr'].unique()

array([ 4, 24], dtype=int64)

In [6]:
df9_drop_columns = ['codesum', 'station_nbr', 'date', 'sunrise', 'sunset', 'depart' ]
df9 = df9.drop(columns = df9_drop_columns)

In [8]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df9_columns = df9.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df9_columns:
    if (i == 'resultdir'):
        df9[i].fillna(df9[i].mode()[0], inplace=True)
        print(df9[i].mode()[0])
    else:
        df9[i].fillna(df9[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df9['relative_humility'] = 100*(np.exp((17.625*((df9['dewpoint']-32)/1.8))/(243.04+((df9['dewpoint']-32)/1.8)))/np.exp((17.625*((df9['tavg']-32)/1.8))/(243.04+((df9['tavg']-32)/1.8))))

# 체감온도 계산
df9["windchill"] = 35.74 + 0.6215*df9["tavg"] - 35.75*(df9["avgspeed"]**0.16) + 0.4275*df9["tavg"]*(df9["avgspeed"]**0.16)

df9 = df9[df9['units'] != 0]

15.0


In [9]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df9 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + sealevel + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df9)
result_df9 = model_df9.fit()

print(result_df9.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.857
Model:                            OLS   Adj. R-squared:                  0.856
Method:                 Least Squares   F-statistic:                     1087.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:34:08   Log-Likelihood:                -4749.2
No. Observations:                5666   AIC:                             9562.
Df Residuals:                    5634   BIC:                             9775.
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]         -0.0475      0

In [10]:
model_df9 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df9)
result_df9 = model_df9.fit()

print(result_df9.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.857
Model:                            OLS   Adj. R-squared:                  0.856
Method:                 Least Squares   F-statistic:                     1087.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:34:46   Log-Likelihood:                -4749.2
No. Observations:                5666   AIC:                             9562.
Df Residuals:                    5634   BIC:                             9775.
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [11]:
anova_result_df9 = sm.stats.anova_lm(result_df9).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df9[anova_result_df9['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(resultspeed),1.0,1.488488,1.488488,4.728427,2.970948e-02
scale(tmax),1.0,2.381813,2.381813,7.566219,5.966235e-03
C(rain),1.0,2.812266,2.812266,8.933625,2.811767e-03
C(other),1.0,8.161155,8.161155,25.925250,3.664349e-07
C(week7),6.0,42.188868,7.031478,22.336646,3.827736e-26
C(item_nbr),10.0,10543.181361,1054.318136,3349.214962,0.000000e+00
C(nothing),2.0,44103.505216,22051.752608,70051.018982,0.000000e+00


In [12]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df9.values, i) for i in range(df9.shape[1])]
vif["features"] = df9.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# resultspeed, C(other), C(week7), C(item_nbr)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1386: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,VIF Factor,features
0,1.063075,store_nbr
1,1.200269,resultdir
2,1.305029,preciptotal
3,1.499458,other
4,1.520370,units
5,1.538268,item_nbr
6,2.696257,week7
7,2.696986,weekend
8,4.472392,relative_humility
9,7.245193,resultspeed


In [13]:
# resultdir, preciptotal, C(rain), C(week7), C(item_nbr)
model_df9 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(other) + C(week7)+ resultspeed + 0', data = df9)
result_df9 = model_df9.fit()

print(result_df9.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.856
Model:                            OLS   Adj. R-squared:                  0.856
Method:                 Least Squares   F-statistic:                     1865.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:36:28   Log-Likelihood:                -4763.2
No. Observations:                5666   AIC:                             9564.
Df Residuals:                    5647   BIC:                             9691.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
C(item_nbr)[6]       3.5442      0.029  

In [15]:
# resultdir, preciptotal, C(rain), C(week9), item_nbr, weekend
X9 = df9[['item_nbr', 'other', 'week7', 'resultspeed']]
y9 = df9['units']
model9 = LinearRegression()

cv9 = KFold(n_splits=10, shuffle=True, random_state=0)

cross_val_score(model9, X9, y9, scoring="r2", cv=cv9)

array([0.28918441, 0.31236185, 0.32410017, 0.31027937, 0.27296166,
       0.3230111 , 0.31061359, 0.26294874, 0.32970757, 0.3104927 ])